In [33]:
import sys
import os
import pandas as pd
from pathlib import Path
import re


def replace_cname(df, oldname, newname):
    new_names = { c: c.replace(oldname, newname) for c in df.columns if oldname in c}
    return df.rename(columns=new_names)

In [57]:
grid_search = pd.DataFrame()

RES_PATH = Path(f"../res/")
files = [os.path.join(path, name) for path, subdirs, files in os.walk(RES_PATH) for name in files]
grid_files = filter(lambda x: "gridSearch" in x, files)

for f in grid_files:
    if "gridSearch" not in f:
        continue
    df = pd.read_pickle(RES_PATH / f)
    
    additional_features = f.replace(".pkl", "").split("__")[1:]
    additional_features = map(lambda x: x.split("_"), additional_features)
    
    for f, v in additional_features:
        df[f"param_{f}"] = v
        
    grid_search = pd.concat([grid_search, df], ignore_index=True)

In [59]:
best_results = grid_search.sort_values("mean_test_score", ascending=True)
best_results = best_results.groupby("param_discretized").head(5)

best_results = best_results.drop(columns=["params"])
features_cols = [c for c in best_results.columns if "param" in c]
best_results = best_results[features_cols + ["mean_test_score"]]

best_results = replace_cname(best_results, "_", " ")
best_results = replace_cname(best_results, "param", "")
best_results = replace_cname(best_results, "module", "")

In [60]:
best_results

,batch size,lr,input size,latent size,rnn layers,rnn size,overlapping,ctxlen,discretized,features,mean test score
0,1024,0.0005,19,32,1,32,0.95,80,True,MAGIK,0.046044
6,1024,0.0005,19,64,1,32,0.95,80,True,MAGIK,0.046929
35,1024,0.0001,19,128,3,128,0.95,80,True,MAGIK,0.047834
4,1024,0.0005,19,32,3,64,0.95,80,True,MAGIK,0.047917
28,1024,0.0001,19,64,3,64,0.95,80,True,MAGIK,0.047974
115,4096,0.0005,19,128,3,128,0.95,80,False,MAGIK,0.090240
123,4096,0.0001,19,128,3,128,0.95,80,False,MAGIK,0.090437
135,6000,0.0001,19,64,3,128,0.95,80,False,MAGIK,0.090527
110,4096,0.0005,19,64,3,64,0.95,80,False,MAGIK,0.090602
111,4096,0.0005,19,64,3,128,0.95,80,False,MAGIK,0.090676
